In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sub=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import re
from sklearn.feature_extraction.text import TfidfVectorizer


In [4]:
def clean_text(text):
    text=text.lower()
    text=re.sub(r"https\S+","",text)
    text=re.sub(r"[^a-z\s]","",text)
    return (text)


In [5]:
train.keyword=train.keyword.fillna("unknown")
train.location=train.location.fillna("unknown")
test.keyword=test.keyword.fillna("unknown")
test.location=test.location.fillna("unknown")
train['text']=train['text'].apply(clean_text)
test['text']=test['text'].apply(clean_text)
test_main=test[['keyword','location','text']]


In [6]:


preprocessor=ColumnTransformer(transformers=[
    ('text',TfidfVectorizer(),'text'),
    ('location',TfidfVectorizer(),'text'),
    ('keyword',TfidfVectorizer(),'keyword')
])
pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',MultinomialNB())
])
X=train[['keyword','location','text']]
y=train['target']


In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

pipeline.fit(X_train,y_train)
predict=pipeline.predict(X_test)
print("----------classification report\n",classification_report(predict,y_test))
print("----------accuracy score\n",accuracy_score(predict,y_test))
main_predict=pipeline.predict(test_main)


----------classification report
               precision    recall  f1-score   support

           0       0.88      0.79      0.83      1218
           1       0.69      0.81      0.75       686

    accuracy                           0.80      1904
   macro avg       0.79      0.80      0.79      1904
weighted avg       0.81      0.80      0.80      1904

----------accuracy score
 0.7998949579831933


In [8]:
submission=pd.DataFrame({
    "id":test.id,
    "target":main_predict
})
submission.to_csv("submissions.csv",index=False)
